# Desafio - Aula 3

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
import warnings

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from unicodedata import normalize

nltk.download('wordnet')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 1) Crie uma classe de tratamento de texto. A classe deverá conter:
   ## -> remoção de números
   ## -> passar o texto para caixa baixa
   ## -> remoção de caracteres especiais
   ## -> remoção de stop-words
   ## -> Stemização/lemmatização (deve ser passado como parâmetro qual abordagem utilizar)

In [2]:
def trata_caracteres(txt):
    txt=[normalize('NFKD',x).encode('ASCII','ignore').decode('ASCII') for x in txt]
    txt=[re.sub('[^a-z|\ ]','',str.lower(x)) for x in txt]
    return txt

In [3]:
stop_words = set(stopwords.words('english'))
def remove_stop_words(txt):
    texto = CountVectorizer(ngram_range=(1,1),stop_words=stop_words)
    texto.fit(txt)
    return texto

In [4]:

def tratar_texto(txt,metodo=''):
    txt = trata_caracteres(txt)
    try:
        texto = remove_stop_words(txt)
        lista_txt = texto.get_feature_names()
    except:
        lista_txt = txt
    if metodo=='lemma':
        lem = WordNetLemmatizer()
        for part_of_speech in ['a', 's', 'r', 'n', 'v']:
            tms = [lem.lemmatize(a,part_of_speech) for a in lista_txt]
    if metodo == 'stem':
        ps = PorterStemmer()
        tms = [ps.stem(a) for a in lista_txt]
    if metodo == '':
        tms = txt
    return " ".join(tms)

## 2) Refaça o exercício de aula (movie_review) realizando o tratamento do texto antes. Houve diferença? Descreva

In [5]:
data = pd.read_csv('movie_review1.csv',index_col=0)

In [6]:
data['sw'] = data.text.apply(lambda x: tratar_texto([x]))

## Texto Puro

In [7]:
vec = CountVectorizer()
vector = vec.fit_transform(data.text)
X_train, X_test, y_train, y_test = train_test_split(vector, data.tag, test_size=0.3, random_state=69)

In [8]:
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)

Resultados da predição sem Tratamento do Texto

In [9]:
print(classification_report(y_test, predicted))

             precision    recall  f1-score   support

          0       0.70      0.71      0.71      9561
          1       0.72      0.70      0.71      9855

avg / total       0.71      0.71      0.71     19416



Matriz de confusão de resultados com texto puro

In [10]:
print(pd.crosstab(y_test,predicted,rownames=['Real'],colnames=['Predito'],margins=False))

Predito     0     1
Real               
0        6828  2733
1        2938  6917


Acurácia da predição sem tratamento de texto 

In [11]:
print(np.mean(predicted == y_test))

0.7079213020189534


## Eliminados caracteres especiais e Stop Words

In [12]:
vec_sw = CountVectorizer()
vector_sw = vec.fit_transform(data.sw)
X_train_sw, X_test_sw, y_train_sw, y_test_sw = train_test_split(vector_sw, data.tag, test_size=0.3, random_state=69)

In [13]:
model_sw = MultinomialNB().fit(X_train_sw, y_train_sw)
predicted_sw = model_sw.predict(X_test_sw)

Resultados com Tratamento de texto 'Stop Words'

In [14]:
print(classification_report(y_test_sw, predicted_sw))

             precision    recall  f1-score   support

          0       0.70      0.70      0.70      9561
          1       0.71      0.70      0.71      9855

avg / total       0.70      0.70      0.70     19416



Resultados das avaliações com tratamento de texto 'Stop Words'

In [15]:
print(pd.crosstab(y_test_sw,predicted_sw,rownames=['Real'],colnames=['Predito'],margins=False))

Predito     0     1
Real               
0        6722  2839
1        2931  6924


Acurária da Predição com Tratamento de texto 'Stop Words' 

In [16]:
print(np.mean(predicted_sw == y_test_sw))

0.7028224145035022


## Utilização da Técnica de Stemização

In [17]:
data['stem'] = data.text.apply(lambda x: tratar_texto([x],'stem'))

In [18]:
vec_stem = CountVectorizer()
vector_stem = vec_stem.fit_transform(data.stem)
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(vector_stem, data.tag, test_size=0.3, random_state=69)

In [19]:
model_stem = MultinomialNB().fit(X_train_stem, y_train_stem)
predicted_stem = model_stem.predict(X_test_stem)

Resultados das avaliações com tratamento de texto 'Stemizado'

In [20]:
print(classification_report(y_test_stem, predicted_stem))

             precision    recall  f1-score   support

          0       0.70      0.70      0.70      9561
          1       0.71      0.71      0.71      9855

avg / total       0.70      0.70      0.70     19416



Matriz de confusão de resultados 'Stemizados'

In [21]:
print(pd.crosstab(y_test_stem,predicted_stem,rownames=['Real'],colnames=['Predito'],margins=False))

Predito     0     1
Real               
0        6675  2886
1        2881  6974


Acurária da Predição com Tratamento de texto 'Stemizado'

In [22]:
print(np.mean(predicted_stem == y_test_stem))

0.7029769262463947


## Utilização da técnica de Lematização

In [23]:
data['lemma'] = data.text.apply(lambda x: tratar_texto([x],'lemma'))

Avaliações com Tratamento de texto 'Lemmatizado'

In [24]:
vec_lemma = CountVectorizer()
vector_lemma = vec.fit_transform(data.lemma)
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(vector_lemma, data.tag, test_size=0.3, random_state=69)

In [25]:
model_lemma = MultinomialNB().fit(X_train_lemma, y_train_lemma)
predicted_lemma = model_lemma.predict(X_test_lemma)

In [26]:
print(classification_report(y_test_lemma, predicted_lemma))

             precision    recall  f1-score   support

          0       0.69      0.70      0.70      9561
          1       0.71      0.70      0.70      9855

avg / total       0.70      0.70      0.70     19416



Acurária da Predição com Tratamento de texto 'Lematizado'

In [27]:
print(np.mean(predicted_lemma == y_test_lemma))

0.7008137618459003


In [28]:
print(pd.crosstab(y_test_lemma,predicted_lemma,rownames=['Real'],colnames=['Predito'],margins=True))

Predito     0     1    All
Real                      
0        6698  2863   9561
1        2946  6909   9855
All      9644  9772  19416


Foram obtidos melhores resultados considerando o texto puro, sem nenhum tratamento de stop words.

## 3) Utilizando o dataset disposto no portal, faça:
   ## -> extraia o dataset na pasta do notebook
   ## -> crie uma função que leia o conteúdo de cada uma das pastas, amazene num dataframe com duas colunas (review, tag)
   ## -> utilize a classe de tratamento de texto criada acima para tratar o texto
   ## -> crie um pipeline de classificação de texto (countvectorizer/tfidfvectorizer,divisão em treino/teste,instância de modelo, fit e predict)
   ## -> imprima o relatório de classificação
   ## -> OBS: teste várias opções de stemming/lemmatizer

In [29]:
import os

criticas = [os.path.join(".\\review_polarity.tar(1)\\review_polarity\\txt_sentoken\\neg\\", nome) for nome in os.listdir(".\\review_polarity.tar(1)\\review_polarity\\txt_sentoken\\neg\\")]
elogios = [os.path.join(".\\review_polarity.tar(1)\\review_polarity\\txt_sentoken\\pos\\", nome) for nome in os.listdir(".\\review_polarity.tar(1)\\review_polarity\\txt_sentoken\\pos\\")]


In [30]:
df = pd.DataFrame()  

In [31]:
def leitura_arquivos(listas,df,tag):
    for lista in listas:
        df = df.append(pd.read_csv(lista,sep='\newline',header=None,names=["review"],index_col=False))
        df['tag'] = tag
    return df

In [32]:
df_positivos = leitura_arquivos(elogios,df,1)

In [33]:
df_negativos = leitura_arquivos(criticas,df,0)

In [34]:
df_final = df_positivos.append(df_negativos)

In [35]:
df_stem = df_final.copy()
df_stem["review"] = df_stem.review.apply(lambda x: tratar_texto([x],'stem'))

In [36]:
df_lemma = df_final.copy()
df_lemma["review"] = df_lemma.review.apply(lambda x: tratar_texto([x],'lemma'))

In [37]:
def analiseTfidVetorizer(df):
    vectfidf = TfidfVectorizer()
    vetortfidf = vectfidf.fit_transform(df.review)
    X_train, X_test, y_train, y_test = train_test_split(vetortfidf, df.tag, test_size=0.3, random_state=42)
    model_stem = MultinomialNB().fit(X_train, y_train)
    predicted = model_stem.predict(X_test)
    print('------------------------------------------------------')
    print('Classificação')
    print(classification_report(y_test, predicted))
    print('------------------------------------------------------')
    print("Matriz de Confusão")
    print(pd.crosstab(y_test,predicted,rownames=['Real'],colnames=['Predito'],margins=False))
    print('------------------------------------------------------')
    print('Acurácia')
    print(np.mean(predicted == y_test))
    print('------------------------------------------------------')


In [38]:
def analiseCountVetorizer(df):
    vectfidf = CountVectorizer()
    vetortfidf = vectfidf.fit_transform(df.review)
    X_train, X_test, y_train, y_test = train_test_split(vetortfidf, df.tag, test_size=0.3, random_state=42)
    model_stem = MultinomialNB().fit(X_train, y_train)
    predicted = model_stem.predict(X_test)
    print('------------------------------------------------------')
    print('Classificação')
    print(classification_report(y_test, predicted))
    print('------------------------------------------------------')
    print("Matriz de Confusão")
    print(pd.crosstab(y_test,predicted,rownames=['Real'],colnames=['Predito'],margins=False))
    print('------------------------------------------------------')
    print('Acurácia')
    print(np.mean(predicted == y_test))
    print('------------------------------------------------------')


## Relatórios de Classificações

### TfidVetorizer

In [39]:
analiseTfidVetorizer(df_final)

------------------------------------------------------
Classificação
             precision    recall  f1-score   support

          0       0.71      0.68      0.70      9556
          1       0.70      0.74      0.72      9860

avg / total       0.71      0.71      0.71     19416

------------------------------------------------------
Matriz de Confusão
Predito     0     1
Real               
0        6473  3083
1        2581  7279
------------------------------------------------------
Acurácia
0.7082818294190358
------------------------------------------------------


In [40]:
analiseTfidVetorizer(df_stem)

------------------------------------------------------
Classificação
             precision    recall  f1-score   support

          0       0.72      0.66      0.69      9556
          1       0.70      0.75      0.72      9860

avg / total       0.71      0.71      0.71     19416

------------------------------------------------------
Matriz de Confusão
Predito     0     1
Real               
0        6343  3213
1        2489  7371
------------------------------------------------------
Acurácia
0.7063246806757314
------------------------------------------------------


In [41]:
analiseTfidVetorizer(df_lemma)

------------------------------------------------------
Classificação
             precision    recall  f1-score   support

          0       0.71      0.67      0.69      9556
          1       0.70      0.74      0.72      9860

avg / total       0.70      0.70      0.70     19416

------------------------------------------------------
Matriz de Confusão
Predito     0     1
Real               
0        6368  3188
1        2566  7294
------------------------------------------------------
Acurácia
0.703646477132262
------------------------------------------------------


### CountVetorizer

In [42]:
analiseCountVetorizer(df_stem)

------------------------------------------------------
Classificação
             precision    recall  f1-score   support

          0       0.70      0.70      0.70      9556
          1       0.71      0.71      0.71      9860

avg / total       0.71      0.71      0.71     19416

------------------------------------------------------
Matriz de Confusão
Predito     0     1
Real               
0        6732  2824
1        2894  6966
------------------------------------------------------
Acurácia
0.7055006180469716
------------------------------------------------------


In [43]:
analiseCountVetorizer(df_final)

------------------------------------------------------
Classificação
             precision    recall  f1-score   support

          0       0.69      0.72      0.71      9556
          1       0.72      0.69      0.71      9860

avg / total       0.71      0.71      0.71     19416

------------------------------------------------------
Matriz de Confusão
Predito     0     1
Real               
0        6873  2683
1        3028  6832
------------------------------------------------------
Acurácia
0.705861145447054
------------------------------------------------------


In [44]:
analiseCountVetorizer(df_lemma)

------------------------------------------------------
Classificação
             precision    recall  f1-score   support

          0       0.70      0.70      0.70      9556
          1       0.71      0.70      0.71      9860

avg / total       0.70      0.70      0.70     19416

------------------------------------------------------
Matriz de Confusão
Predito     0     1
Real               
0        6731  2825
1        2939  6921
------------------------------------------------------
Acurácia
0.7031314379892872
------------------------------------------------------
